In [77]:
from threading import Thread
import telebot
import datetime

class Plant:
    def __init__(self, name):
        self.name = name
        self.waslastWatered = datetime.datetime.utcnow()
        self.intervalMin = 1440 
    
    def setInterval(self, interval):
        self.intervalMin = max(0, interval)
    
    def watered(self):
        self.waslastWatered = datetime.datetime.utcnow()

userplants = {}

bot = telebot.TeleBot('')

@bot.message_handler(content_types=['text'])
def start(message):
    if message.text == '/add':
        bot.send_message(message.from_user.id, "Какое растение?");
        bot.register_next_step_handler(message, get_name);     
    else:
        bot.send_message(message.from_user.id, 'Напиши /add');
        
def get_name(message): #получаем имя
    userplants[message.from_user.id] = Plant(message.text)
    bot.send_message(message.from_user.id, 'Как часто поливать?')
    bot.register_next_step_handler(message, get_interval)
    
def get_name(message): #получаем имя
    userplants.pop[message.from_user.id] = Plant(message.text)
    bot.send_message(message.from_user.id, 'Как часто поливать?')
    bot.register_next_step_handler(message, get_interval)

def get_interval(message): #получаем interval 
    try:
        interval = int(message.text) #проверяем, что interval введен корректно
        userplants[message.from_user.id].setInterval(interval)
        bot.send_message(message.from_user.id, 'Готово')
    except Exception:
        bot.send_message(message.from_user.id, 'Цифрами, пожалуйста')
        bot.register_next_step_handler(message, get_interval)
    
def notify():
    while True:
        now = datetime.datetime.utcnow()
        for user, plant in userplants.items():
            if (plant.waslastWatered + datetime.timedelta(minutes = plant.intervalMin)  < now):
                bot.send_message(user, f'Пора полить {plant.name}!')
                plant.watered()

notifythread = Thread(target=notify)        
notifythread.start()    
bot.polling(none_stop=True, interval=0)
